In [1]:
# This notebook produces the plots for the differential cross section results in the paper. In addition, one can plot a decomposed norm-shape covariance matrix.
# Prerequisites:
# 1) Systematic uncertainty toys, in the format of FoldedSysRecoEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+Alpha+str(alpha)".npy"
# Where energy is the gaussian peak, rebin_factor is the width of bins, energy_width is the gaussian RMS and alpha is the regularisation parameter that's been used to construct the VF.
# 2) Statistical uncertainty toys, in the format of FoldedStatRecoEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+alpha_string+".npy"
# 3) Distribution by modes, in the format of OmegaReco"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Mode"+modes[i]+".npy"
# All of which can be produced using NumpyToysToMeasurements

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipynb.fs.full.CoefficientsCalcPlus import GetCoefficientsFlux
from ipynb.fs.full.CoefficientsCalcPlus import get_normalization
import sklearn.linear_model as linear_model
import scipy
import warnings; warnings.simplefilter('ignore')
import os
from matplotlib.pyplot import figure
import random as rnd
import matplotlib.ticker as ticker
import uproot4 as uproot
import scipy.stats
import seaborn as sns
import sklearn
from matplotlib.colors import Normalize

In [3]:
#Helper functions

def rebinned_bin_centers(nbins, xmin, xmax, factor):
    # Compute the bin widths for the original histogram
    bin_width = (xmax - xmin) / nbins
    
    # Compute the bin edges for the original histogram
    edges = np.linspace(xmin, xmax, nbins + 1)
    
    # Compute the new bin widths after rebinning
    new_bin_width = bin_width * factor
    
    # Compute the number of new bins
    new_nbins = int(nbins / factor)
    
    # Compute the bin edges for the rebinned histogram
    new_edges = np.linspace(xmin, xmax, new_nbins + 1)
    
    # Compute the bin centers for the rebinned histogram
    centers = (new_edges[1:] + new_edges[:-1]) / 2
    
    return centers

def slice_matrix(matrix, bounds):
    """
    Slice a matrix based on the given 4-tuple bounds.

    Args:
    matrix (list of lists): The input matrix.
    bounds (tuple): A 4-tuple (start_row, end_row, start_col, end_col) specifying the slice bounds.

    Returns:
    list of lists: The sliced submatrix.
    """
    start_row, end_row, start_col, end_col = bounds
    sliced_matrix = np.array([row[start_col:end_col] for row in matrix[start_row:end_row]])
    return sliced_matrix

In [4]:
# Adjustable parameters (user input)
energies = [625,750,875] #Gaussian peaks in MeV
rebin_factor = 40 #Bin widths
width = 70 #Gaussian RMS in MeV
modes_bool = True #Plot modes
folder_path = "../FluxTest/FlexibleUnfoldingResults/"

In [ ]:
# === Global Plot Settings ===
plt.rcParams['font.size'] = 45
plt.rcParams['text.usetex'] = True
lwidth = 2.75
col = 'mediumvioletred'
lighter_col = (*[(1 + c) / 2 for c in mcolors.to_rgba(col)[:3]], mcolors.to_rgba(col)[3])
bbox_to_anchor = [0.393, 0.902]

# === Data Preparation ===
cv_array, true_array = [], []
shape_unc_array, stat_unc_array = [], []
stat_shape_unc_array, norm_unc_array, total_unc_array = [], [], []
modes_arrays_array = []

# Flags
unfolded = False
hist = True

# Plot configuration
rebin_factor = 40
energies = [750, 875, 1000]
widths = [70, 100, 130]
modes = ['CCQE', 'RES', '2p2h', 'Other']

# Bin definitions
if unfolded:
    energy_bin_centers = rebinned_bin_centers(16000, -8000, 8000, rebin_factor) / 1000
    centers = energy_bin_centers[len(energy_bin_centers) // 2:]
    centers_for_hist = centers - (1e-3 * rebin_factor / 2)
else:
    energy_bin_centers = rebinned_bin_centers(16000, -8000, 8000, rebin_factor) / 1000
    centers = energy_bin_centers
    centers_for_hist = centers - (1e-3 * rebin_factor / 2)

# Loop over energies
for energy_val in energies:
    energy, gaus_width = energy_val, 70
    width = gaus_width
    energy_width = width
    
    # Load solutions
    if unfolded:
        solutions_sys = np.load(f"FlexibleUnfoldingResults/SystematicUnfoldingEnergy{energy}BinWidth{rebin_factor}FluxWidth{energy_width}.npy")
        solutions_stat = np.load(f"FlexibleUnfoldingResults/StatisticalUnfoldingEnergy{energy}BinWidth{rebin_factor}FluxWidth{energy_width}.npy")
    else:
        solutions_sys = np.load(f"FlexibleUnfoldingResults/FoldedSysRecoEnergy{energy}BinWidth{rebin_factor}FluxWidth{energy_width}.npy")
        solutions_stat = np.load(f"FlexibleUnfoldingResults/FoldedStatRecoEnergy{energy}BinWidth{rebin_factor}FluxWidth{energy_width}.npy")
    
    true = np.load(f"FlexibleUnfoldingResults/TrueEnergy{energy}BinWidth{rebin_factor}FluxWidth{energy_width}.npy")
    cv = solutions_sys.mean(axis=1)
    
    # Append data
    cv_array.append(cv)
    true_array.append(true)
    
    if modes:
        modes_arrays_array.append([
            np.load(f"FlexibleUnfoldingResults/OmegaReco{energy}BinWidth{rebin_factor}FluxWidth{energy_width}Mode{mode}.npy")
            for mode in modes
        ])

# === Plotting ===
print("Plotting histograms and uncertainties...")
fig, axes = plt.subplots(nrows=1, ncols=len(energies), figsize=(24, 9), sharex='col', dpi=300)
fig.set_facecolor('white')

for i, ax1 in enumerate(axes):
    ax1.set_xlim(-0.3, 1.2)
    ax1.set_ylim(-0.3, 1.8)
    
    # Plot as histograms
    n, bins, _ = ax1.hist(centers, bins=centers_for_hist, weights=cv_array[i], histtype='step', color='black', linewidth=1, linestyle='dashed', label='Total')
    
    # Plot error bars
    ax1.errorbar(centers, cv_array[i], yerr=shape_unc_array[i], color=col, fmt=' ', linewidth=0.5, capsize=2, capthick=1, ecolor=col)
    ax1.errorbar(centers, cv_array[i], yerr=stat_shape_unc_array[i], color=col, label='Reco \n Shape (inner) $\\bigoplus$ Stat (outer) Unc', fmt='.', linewidth=0.5, capsize=2, capthick=1, ecolor=col)
    
    # Plot modes
    for j, mode in enumerate(modes):
        ax1.hist(centers, bins=centers_for_hist, weights=modes_arrays_array[i][j], histtype='step', linestyle='--', label=mode)
    
    # Fill normalization uncertainty
    ax1.fill_between(centers, 0, norm_unc_array[i], color='lightsteelblue', label=r"Norm Unc")
    
    # Axis labels and titles
    if i == 0:
        ax1.set_ylabel(r'$\frac{d\langle\tilde{\sigma}\rangle}{d\omega_{\mathrm{reco}}}$ [$10^{-38} \frac{\mathrm{cm}^{2}}{\mathrm{GeV}}$]')
    ax1.set_title(r'$\tilde{E}_{\nu} = ' + f"{energies[i]}" + r'\ \mathrm{MeV}$', y=1.02)

# === Legend and Layout Adjustments ===
handles, labels = ax1.get_legend_handles_labels()
legend = fig.legend(handles, labels, fontsize=22, bbox_to_anchor=(0.98, 0.5), loc='center left', borderaxespad=0)

plt.tight_layout()
plt.subplots_adjust(right=0.93)
plt.show()

In [ ]:
# === Global Plot Settings ===
plt.rcParams['font.size'] = 36
plt.rcParams['text.usetex'] = True

# === Configuration ===
titles = ["Statistical Correlation", "Systematic Correlation", "Total Correlation"]
covariance_matrices = []

# === 1️⃣ Compute Covariance Matrices ===
print("Calculating covariance matrices...")
covariance_matrix_stat = np.cov(solutions_stat_array)
covariance_matrix_sys = np.cov(solutions_sys_array)

# Store for plotting
covariance_matrices.append(covariance_matrix_stat)
covariance_matrices.append(covariance_matrix_sys)

# Compute Total Covariance Matrix
covariance_matrix_total = covariance_matrix_stat + covariance_matrix_sys
sqrt_diagonal = np.sqrt(np.diag(covariance_matrix_total))
correlation_matrix_total = covariance_matrix_total / (sqrt_diagonal[:, None] * sqrt_diagonal[None, :])

# === 2️⃣ Compute Correlation Matrices ===
correlation_matrices = [
    np.corrcoef(solutions_stat_array),
    np.corrcoef(solutions_sys_array),
    correlation_matrix_total
]

# === 3️⃣ Plot Correlation Matrices ===
print("Plotting correlation matrices...")
for i, (title, correlation_matrix) in enumerate(zip(titles, correlation_matrices)):
    plt.figure(figsize=(8, 6), dpi=300)
    plt.gcf().set_facecolor('white')
    plt.title(title, fontsize=30)
    
    # Plot the correlation matrix
    im = plt.imshow(correlation_matrix, cmap='coolwarm')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    
    # === Dashed lines for block structure ===
    for j in range(1, len(upper_index)):
        k = (np.array(upper_index[:j]) - 1).sum()
        plt.axvline(k - 0.5, color='black', linestyle='dashed')
        plt.axhline(k - 0.5, color='black', linestyle='dashed')
    
    # === Generate LaTeX tick labels ===
    tick_labels = []
    for j in range(len(upper_index)):
        tick_labels += [f"$\\mathrm{{{round(1e-3 * rebin_factor // 2 + 1e-3 * k * rebin_factor, 2)}}}$" 
                        for k in range(lower_index, upper_index[j])]
    
    # Thin out the labels to avoid overcrowding
    tick_labels = [tick_labels[j] for j in range(0, len(tick_labels), len(energies) + 1)][::2]
    tick_positions = [j - 0.5 for j in range(0, len(correlation_matrix), len(energies) + 1)][::2]
    
    plt.xticks(tick_positions, tick_labels, rotation=45)
    plt.yticks(tick_positions, tick_labels)

    # Label axes
    omega_string = '$\\omega_{\mathrm{reco}}$ [$\mathrm{GeV}$]'
    if unfolded:
        omega_string = '$\\omega_{\mathrm{true}}$ [$\mathrm{GeV}$]'
    plt.xlabel(omega_string, fontsize=30)
    plt.ylabel(omega_string, fontsize=30)
    plt.clim([-1, 1])
    
    # === Layout and Display ===
    plt.tight_layout()
    plt.show()